#### Этап 0. Подготовка окружения

In [3]:
# Автоперезагрузка импортов
%load_ext autoreload
%autoreload 2

from pathlib import Path
import sys
import torch
import os

# Отключаем параллелизм в токенизаторах (иначе могут быть проблемы с многопроцессорностью)
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Подключаем нужные пути
SRC_DIR = Path('src')
MODELS_DIR = Path('models')
SRC_DIR.mkdir(exist_ok=True, parents=True)
MODELS_DIR.mkdir(exist_ok=True, parents=True)
sys.path.append(str(SRC_DIR))

from data_utils import create_dataset_processed, split_dataset
from next_token_dataset import create_loaders
from lstm_model import SimpleLSTMNextToken, count_parameters
from eval_lstm import eval_rouge, autocomplete_examples
from lstm_train import train_one_epoch, validate


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


#### Этап 1. Сбор и подготовка данных

In [4]:
# Очистка датасета и токенизация
n_rows = create_dataset_processed()
print(f"[OK] Создан очищенный датасет, строк: {n_rows}.")

# Разбиение на выборки train/val/test
n_train, n_val, n_test = split_dataset()
print(f"[OK] Созданы тренировочная, валидационная, тестовая выборки: {n_train}, Val: {n_val}, Test: {n_test}.")

# Создание Dataset и DataLoader
(dsets, loaders, tokenizer) = create_loaders(
    data_dir='data',
    tokenizer_name='bert-base-multilingual-cased',
    seq_len=32,
    batch_size=64,
    add_eos=True,
)

train_ds, val_ds, test_ds = dsets
train_loader, val_loader, test_loader = loaders
print(f"[OK] Созданы Dataset и DataLoader для обучения модели:")
print(f"Train batches: {len(train_loader)} | Val batches: {len(val_loader)}")

# Просмотр одного батча
batch = next(iter(train_loader))
for k, v in batch.items():
    try:
        shape = v.shape
    except AttributeError:
        shape = type(v)
    print(k, shape)


[OK] Создан очищенный датасет, строк: 1600498.
[OK] Созданы тренировочная, валидационная, тестовая выборки: 1280398, Val: 160050, Test: 160050.
[OK] Созданы Dataset и DataLoader для обучения модели:
Train batches: 4165 | Val batches: 510
input_ids torch.Size([64, 32])
labels torch.Size([64, 32])


#### Этап 2. Реализация рекуррентной сети

In [6]:
# Создание модели, оптимизатора и функции потерь
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

pad_id = tokenizer.pad_token_id if tokenizer.pad_token_id is not None else 0
model = SimpleLSTMNextToken(
    vocab_size=tokenizer.vocab_size,
    emb_dim=256,
    hidden_dim=128,
    num_layers=1,
    pad_idx=pad_id,
    dropout=0.1,
).to(device)

print("[OK] Модель создана. Кол-во обучаемых параметров:", count_parameters(model))

criterion = torch.nn.CrossEntropyLoss(ignore_index=-100)  # labels у нас уже -100 на паддингах
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-3, weight_decay=1e-2)


[OK] Модель создана. Кол-во обучаемых параметров: 46223227


#### Этап 3. Тренировка модели

In [7]:
# Тренировка (train/val loss + ROUGE на валидации)
from time import time

epochs = 2
log_every = 100
rouge_max_batches = 10   # увеличить для более точной оценки (медленнее)

best_val = float("inf")
best_ckpt_path = MODELS_DIR / "lstm_next_token_best.pt"

print(f"[OK] Начинаем тренировку на {epochs} эпох:")
for epoch in range(1, epochs + 1):
    print(f"\n=== Epoch {epoch}/{epochs} ===")
    t0 = time()

    train_loss = train_one_epoch(model, train_loader, optimizer, criterion, device, log_every=log_every)
    val_loss   = validate(model, val_loader, criterion, device)

    # ROUGE (3/4 → 1/4) на валидации
    rouge = eval_rouge(model, val_loader, tokenizer, pad_id=pad_id, max_batches=rouge_max_batches)

    dt = time() - t0
    print(f"Epoch {epoch}: train loss {train_loss:.4f} | val loss {val_loss:.4f} | time {dt:.1f}s")
    print("ROUGE:", " ".join([f"{k}={v:.4f}" for k, v in rouge.items()]))

    # Сохраняем чекпойнт за эпоху
    ckpt = {
        "state_dict": model.state_dict(),
        "hparams": {
            "emb_dim": 256,
            "hidden_dim": 128,
            "num_layers": 1,
            "dropout": 0.1,
            "seq_len": 32,
            "tokenizer_name": "bert-base-multilingual-cased",
        },
        "metrics": {
            "train_loss": float(train_loss),
            "val_loss": float(val_loss),
            **{f"rouge/{k}": float(v) for k, v in rouge.items()},
        },
        "epoch": epoch,
    }
    ep_path = MODELS_DIR / f"lstm_next_token_e{epoch}_valloss{val_loss:.3f}.pt"
    torch.save(ckpt, ep_path)
    print("Saved:", ep_path)

    if val_loss < best_val:
        best_val = val_loss
        torch.save(ckpt, best_ckpt_path)
        print("→ Updated best:", best_ckpt_path)

print(f"[OK] Модель обучена.")


[OK] Начинаем тренировку на 2 эпох:

=== Epoch 1/2 ===
  step   100 | loss 7.0981
  step   200 | loss 6.7759
  step   300 | loss 6.3413
  step   400 | loss 6.0904
  step   500 | loss 6.0971
  step   600 | loss 5.6508
  step   700 | loss 5.5767
  step   800 | loss 5.2049
  step   900 | loss 5.5405
  step  1000 | loss 5.3453
  step  1100 | loss 5.3905
  step  1200 | loss 5.2709
  step  1300 | loss 5.0088
  step  1400 | loss 4.8302
  step  1500 | loss 5.2144
  step  1600 | loss 4.9991
  step  1700 | loss 5.0433
  step  1800 | loss 4.9426
  step  1900 | loss 5.1331
  step  2000 | loss 4.9505
  step  2100 | loss 5.1774
  step  2200 | loss 5.1229
  step  2300 | loss 5.0882
  step  2400 | loss 4.9805
  step  2500 | loss 5.1598
  step  2600 | loss 5.0963
  step  2700 | loss 4.9539
  step  2800 | loss 4.9368
  step  2900 | loss 4.8557
  step  3000 | loss 4.7712
  step  3100 | loss 4.7384
  step  3200 | loss 4.7804
  step  3300 | loss 4.7190
  step  3400 | loss 4.8225
  step  3500 | loss 4.8961


#### Этап 3. Тестовая метрика и примеры автодополнений

In [8]:
# Загружаем лучший чекпойнт (если нужно)
state = torch.load(best_ckpt_path, map_location="cpu")
model.load_state_dict(state["state_dict"])
model.to(device).eval()

# ROUGE на тесте
r_test = eval_rouge(model, test_loader, tokenizer, pad_id=pad_id, max_batches=rouge_max_batches)
print("[OK] ROUGE метрика:")
for k, v in r_test.items():
    print(f"  {k}: {v:.4f}")

# Несколько примеров (FULL / PREFIX / TARGET / PRED)
print("[OK] Примере автозаполнений:")
autocomplete_examples(model, test_loader, tokenizer, pad_id=pad_id, num_examples=5)


[OK] ROUGE метрика:
  rouge1: 0.1184
  rouge2: 0.0402
  rougeL: 0.1151
  rougeLsum: 0.1153
[OK] Примере автозаполнений:
————————————————————————————————————————————————————————————
FULL:    errr morning s nice day sun is shining i have the entire day and night off woo really bad tooth ache from brace oh
PREFIX:  errr morning s nice day sun is shining i have the entire day and night off woo really bad too
TARGET:  ##th ache from brace oh
PRED:    much i m so tired i m
————————————————————————————————————————————————————————————
FULL:    ##rr morning s nice day sun is shining i have the entire day and night off woo really bad tooth ache from brace oh well
PREFIX:  ##rr morning s nice day sun is shining i have the entire day and night off woo really bad tooth
TARGET:  ache from brace oh well
PRED:    ##ng myself i m so tired
————————————————————————————————————————————————————————————
FULL:    morning s nice day sun is shining i have the entire day and night off woo really bad tooth ache 

#### Этап 4. Использование предобученного трансформера

In [ ]:
# Оценка качества предобученного distilgpt2 (трансформер)
from eval_transformer_pipeline import eval_transformer_rouge

rouge_scores = eval_transformer_rouge(
    data_path="data/val.csv",
    text_col="text",
    max_samples=400
)

print("[OK] Оценка distilgpt2 завершена.")

Device set to use cpu


[RUN] distilgpt2 на 400 примерах (CPU).
[OK] Метрики ROUGE (distilgpt2):
  rouge1: 0.0874
  rouge2: 0.0150
  rougeL: 0.0870
  rougeLsum: 0.0870
[OK] Пример автозаполнений:
————————————————————————————————————————————————————————————
FULL:   nice hair cut dude why were your students leaving in the middle of class 1st period
PREFIX: nice hair cut dude why were your students leaving in the middle
TARGET: of class 1st period
PRED  : of a school day
————————————————————————————————————————————————————————————
FULL:   i have to know what you are coming up with for the ai tour
PREFIX: i have to know what you are coming up with
TARGET: for the ai tour
PRED  : .
————————————————————————————————————————————————————————————
FULL:   i wanna see the new dress even though i didn t see the first dress
PREFIX: i wanna see the new dress even though i didn t
TARGET: see the first dress
PRED  : ...
————————————————————————————————————————————————————————————
FULL:   hates payer contracts wasted energy or

---
---

#### **Этап 5. Формулирование выводов**

##### **Метрики ROUGE**
| Модель | ROUGE-1 | ROUGE-2 | ROUGE-L | ROUGE-Lsum |
|:--|:--:|:--:|:--:|:--:|
| **LSTM (моя модель)** | **0.1184** | **0.0402** | **0.1151** | **0.1153** |
| **DistilGPT-2 (на 400 примерах)** | 0.0874 | 0.0150 | 0.0870 | 0.0870 |


##### **Примеры автозаполнений**

**LSTM (моя модель):**
> **PREFIX:** “nice day sun is shining i have the entire day and night off woo really bad tooth ache from”  
> **PRED:** “the beach and i m so tired”  
> **TARGET:** “brace oh well tweet me”  
Грамматически связна и стабильно дописывает последовательности, но часто повторяет шаблонные фразы (“i m so tired”, “i m so”).

**DistilGPT-2:**
> **PREFIX:** “nice hair cut dude why were your students leaving in the middle”  
> **PRED:** “of a school day”  
> **TARGET:** “of class 1st period”  
Коротко и ближе к смыслу, но иногда выдаёт неполные или случайные окончания (*“.”*, *“...”*, *“or a ch”*).


##### **Сравнение моделей**

| Критерий | **LSTM** | **DistilGPT-2** |
|:--|:--|:--|
| **Точность (ROUGE)** | Выше — ближе к эталонным продолжениям | Ниже, но улучшается при увеличении кол-ва примеров |
| **Смысловая связность** | Средняя, часто повторяет шаблонные фразы | Лучше улавливает контекст, но нестабильна |
| **Грамматика и структура** | Стабильная и правильная | Иногда обрывает фразы |
| **Ресурсы (CPU, память)** | Очень лёгкая и быстрая | Требует больше вычислений |


##### **Рекомендации по применению**

| Сценарий | Рекомендуемая модель | Причина |
|-----------|---------------------|----------|
| **Автодополнение в мобильных приложениях** | **LSTM** | Компактна, быстра, стабильно предсказывает текст, подходит для работы на CPU |
| **Креативные и контекстные генерации (чат-боты, письма)** | **DistilGPT-2** | Более гибкая, естественная речь |


##### **Итог**

> **LSTM-модель** лучше подходит для **мобильных и офлайн-приложений**, где важна стабильность, компактность и скорость.  
> **DistilGPT-2** эффективнее для **креативных и контекстных задач**, особенно при дальнейшем обучениина.
